# ETL

La primera columna "node" contiene el nombre del nodo en el que se toman los datos (eg c7102). Para los datos de las enfriadoras, aparecen en esta columna como "1" y "2" (enfriadora 1, enfriadora 2 respectivamente) para los datos de temperatura y presión y como "basement" para los datos de consumo (la enfriadora 1 se corresponde con la columna "Power13" y la 2 con "Power14").

La segunda y tercera columna incluyen los rangos de fecha para las series temporales de esa fila. Como se indicó antes, de 2018/01/01 hasta 2021/01/06.

La columna 4 "power": incluye los consumos de los nodos (ie node="cxxxx"). En cada celda se encuentra una serie temporal en formato diccionario en el que la key se corresponde con el timestamp del dato.

Las columnas 5 y 6 'Power13', 'Power14' se corresponden con el consumo de las enfriadoras (medido en los cuadros PM13 PM14) como se indicó en la explicación de la columna nodo. Solo debería tener datos para node="basement". En cada celda se encuentra una serie temporal en formato diccionario en el que la key se corresponde con el timestamp del dato.

Las columnas de la 7 a 10 :'in' -(free cooling)-> 'evaporator' -(compresores)-> 'out'. Y  'ambient' sería la temperatura externa. Estas incluyen las medidas de las temperaturas en las enfriadoras (respectivamente de entrada, salida, en el evaporador y ambiente). Solo debería tener datos para node="1" o "2" dependiendo de que enfriadora se trate. En cada celda se encuentra una serie temporal en formato diccionario en el que la key se corresponde con el timestamp del dato.

Las columnas 11, 12: 'Compressor1', 'Compressor2' son equivalentes al punto 
anterior pero con los datos de presión en cada uno de los dos compresores de cada enfriadora. Solo debería tener datos para node="1" o "2" dependiendo de que enfriadora se trate. En cada celda se encuentra una serie temporal en formato diccionario en el que la key se corresponde con el timestamp del dato.

## Objetivos:
Como series temporales usaremos (agrupando cada 30 minutos):
### Suma del consumo de los nodos -  Originalmente en Wattios
### Suma del consumo de las dos enfriadoras - KW (Power 13 enfriadora 1, Power 14 enfriadora 2)
### Máximo de la presión de los 4 compresores - En Pascales (2 enfriadoras, 4 compresores, una lista con dos diccionarios en principio)
### Número de compresores activos (compresores con presión mayor a 15 bars) 
### Cogeremos las temperaturas in, evaporator, out, ambient correspondientes a la enfriadora activa (la que tenga consumo mayor a 10KW) (referencia, agua caliente que volver del CPD es en torno a 18 grados)
### Cogeremos la diferencia entre la temperatura ambient y el setpoint (se puede obtener como media de temperatura out) 

En total tenemos 9 series temporales (a determinar si interesa mantener por separado temperatura ambiente y la diferencia de temperatura entre ambiente y setpoint).

Trataremos de hacer:
- CU1: Predicción de la suma del consumo de las enfriadoras a 24h (será función de cuanto free cooling se pueda utilizar)
- CU2: Predicción de la presión máxima a 24h

Para el entrenamiento del CU1 podemos usar datos generales aunque serán más significativos los de invierno (sólo se puede usar free cooling aquellos momentos en que la temperatura ambiente es menor a la temperatura in, en verano esto solo es probable que ocurra durante la noche).

Para el entrenamiento del CU2 son sólo relevantes los datos de los meses de verano.


Para visualizar los datos se puede usar el siguiente dashboard:

Dashboard:

http://grafana.srv.cesga.es/d/000000016/dcim?orgId=1


In [2]:
## Imports
import os
import copy
import statsmodels
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
#Spark dependencies
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark import StorageLevel, SparkConf
from pyspark.ml.feature import StandardScaler, VectorAssembler, PCA
from pyspark.mllib.linalg import SparseVector, DenseVector, VectorUDT
from pyspark.ml.classification import LogisticRegression
#Other configs
pd.options.display.float_format = '{:.2f}'.format

#Useful directory variables
src_path = os.getcwd()
root_path = os.path.dirname(src_path)
data_path = root_path+"/datasets"
visualization_path = root_path+"/data_visualization"

Primer Paso: pasar de una tabla con una unica fila conteniendo las series temporales como diccionarios en una celda (o una lista con dos diccionarios si hay dos series - Compresor 1 y 2 de Enfriadora 1 y lo mismo para enfriadora 2, a series temporales tabulares

In [3]:
df = spark.read.parquet("output_final.parquet") #Functional programming. Reading the raw data file with the Structured API
df.printSchema()

root
 |-- node: string (nullable = true)
 |-- start_time: timestamp (nullable = true)
 |-- end_time: timestamp (nullable = true)
 |-- power: map (nullable = true)
 |    |-- key: timestamp
 |    |-- value: float (valueContainsNull = true)
 |-- Power13: map (nullable = true)
 |    |-- key: timestamp
 |    |-- value: float (valueContainsNull = true)
 |-- Power14: map (nullable = true)
 |    |-- key: timestamp
 |    |-- value: float (valueContainsNull = true)
 |-- in: map (nullable = true)
 |    |-- key: timestamp
 |    |-- value: float (valueContainsNull = true)
 |-- out: map (nullable = true)
 |    |-- key: timestamp
 |    |-- value: float (valueContainsNull = true)
 |-- evaporator: map (nullable = true)
 |    |-- key: timestamp
 |    |-- value: float (valueContainsNull = true)
 |-- ambient: map (nullable = true)
 |    |-- key: timestamp
 |    |-- value: float (valueContainsNull = true)
 |-- Compressor1: map (nullable = true)
 |    |-- key: timestamp
 |    |-- value: float (valueContains

In [4]:
df.createOrReplaceTempView("df")
spark.sql("SELECT node from df").show(291)